In [2]:
from os.path import join
import pandas as pd

import gensim
from gensim.corpora import Dictionary, MmCorpus
from gensim.models import TfidfModel, LdaModel, LdaMulticore
from gensim.models.phrases import Phrases, Phraser

import pyLDAvis as ldavis
import pyLDAvis.gensim
ldavis.enable_notebook()

import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from constants import FULL_PATH, NLP_PATH, SMPL_PATH, POS, NOUN, PROPN, TOKEN, HASH, SENT_IDX

def docs_to_lists(token_series):
    return tuple(token_series.tolist())

In [3]:
dataset = ['OnlineParticipation', 'FAZ'][1]
df = pd.read_pickle(join(SMPL_PATH, dataset + '_simple.pickle'))
df = df[df[POS].isin({NOUN, PROPN, 'NER', 'NPHRASE'})]
df = df.groupby([HASH])[TOKEN].agg(docs_to_lists)

In [4]:
texts = df.values
dictionary = Dictionary(texts)
dictionary.filter_extremes(no_below=2, no_above=0.5)
# filter some noice (e.g. special characters)
# filter_tokens(bad_ids=None, good_ids=None)
bow_corpus = [dictionary.doc2bow(text) for text in texts]
tfidf = TfidfModel(bow_corpus)
tfidf_corpus = tfidf[bow_corpus]

In [6]:
corpus = [bow_corpus, tfidf_corpus][1]
MmCorpus.serialize('../data/{}.mm'.format(dataset), corpus)
corpus_fake = MmCorpus('../data/{}.mm'.format(dataset))

In [7]:
#ldamodel = LdaModel(
ldamodel = LdaMulticore(
    random_state=42,
    # corpus=bow_corpus, 
    corpus=tfidf_corpus,
    num_topics=100, 
    id2word=dictionary, 
    alpha=0.1,  # 'auto' not in multicore implementation 
    eta=0.001,  # 'auto',
    # gamma_threshold
    # minimum_phi_value
    # offset
    # decay
    # update_every=1, # not in multicore implementation 
    chunksize=10000, 
    passes=1,
)
ldamodel.print_topics(num_topics=100, num_words=10)

prepared_data = ldavis.gensim.prepare(ldamodel, corpus_fake, dictionary)
prepared_data

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
71     1.340305        1       1 -0.015561  0.016376
75     1.290655        1       2 -0.012905  0.013996
49     1.284795        1       3 -0.007993  0.013588
30     1.227614        1       4 -0.020040  0.006081
10     1.220969        1       5 -0.016017 -0.003541
44     1.218435        1       6  0.006547  0.005087
15     1.214530        1       7  0.001013 -0.011170
39     1.170513        1       8  0.029536 -0.003823
41     1.169193        1       9  0.008260 -0.012272
34     1.161943        1      10 -0.025670  0.017543
16     1.144258        1      11 -0.009445  0.010147
95     1.132890        1      12 -0.016678 -0.004935
22     1.130967        1      13 -0.007155  0.010479
97     1.120250        1      14  0.014668 -0.011780
36     1.114080        1      15 -0.010894  0.009754
27     1.099788        1      16 -0.029120 -0.018765
73     1.097312        1      17  0.002526  0.012448
14     1.089003        1      18 -0.010079 -0.008546
67     1.088964        1      19 -0.014674  0.000828
51     1.083605        1      20 -0.000922  0.008758
62     1.080211        1      21  0.019387  0.012849
58     1.060547        1      22 -0.012990  0.004341
54     1.060278        1      23 -0.014944  0.008994
4      1.058173        1      24 -0.031978  0.011582
93     1.053009        1      25 -0.022795  0.014225
89     1.046717        1      26  0.004699  0.003796
56     1.046314        1      27 -0.000412 -0.004114
81     1.041515        1      28 -0.013509 -0.015127
42     1.038152        1      29  0.001638 -0.019365
79     1.033806        1      30  0.031303 -0.019354
...         ...      ...     ...       ...       ...
1      0.941179        1      71 -0.005362 -0.008851
70     0.938088        1      72  0.003331 -0.023800
23     0.935688        1      73  0.130483  0.030469
96     0.931916        1      74 -0.010735 -0.027240
57     0.930579        1      75 -0.015425  0.009602
50     0.924856        1      76 -0.035664 -0.003521
68     0.916369        1      77 -0.019180  0.018672
61     0.914547        1      78 -0.022651  0.011422
77     0.913956        1      79  0.048831 -0.036696
72     0.913135        1      80  0.005206  0.020832
5      0.911664        1      81 -0.025885  0.040516
83     0.909782        1      82  0.014625 -0.031160
40     0.908722        1      83 -0.012730  0.044476
29     0.900814        1      84 -0.030618 -0.004817
82     0.894972        1      85 -0.001841  0.014183
74     0.894234        1      86  0.002454 -0.008241
33     0.890681        1      87 -0.004329  0.028224
2      0.883806        1      88  0.004164 -0.006238
53     0.878483        1      89 -0.036640 -0.019475
32     0.877588        1      90 -0.006244  0.017724
55     0.875701        1      91 -0.020804 -0.009804
8      0.870991        1      92 -0.013636  0.053974
65     0.865934        1      93 -0.011836  0.008468
13     0.861328        1      94  0.041732  0.060942
60     0.854724        1      95  0.029929  0.020164
6      0.845225        1      96  0.002077  0.007768
37     0.836629        1      97 -0.013517 -0.036484
63     0.833448        1      98 -0.009323 -0.029588
19     0.823039        1      99  0.009989 -0.018969
7      0.820275        1     100  0.006917  0.021949

[100 rows x 5 columns], topic_info=       Category        Freq          Term       Total  loglift  logprob
term                                                                   
5997    Default  256.000000     Nordkorea  256.000000  30.0000  30.0000
2129    Default   53.000000          Opel   53.000000  29.0000  29.0000
1163    Default  142.000000         Apple  142.000000  28.0000  28.0000
7583    Default   77.000000       Bitcoin   77.000000  27.0000  27.0000
1837    Default  157.000000        Israel  157.000000  26.0000  26.0000
11041   Default   41.000000         Cohen   41.000000  25.0000  25.0000
9283    Default   

### Evaluation